# Example Usage for `mix_gamma_vi`

In [8]:
from mix_gamma_vi import mix_gamma_vi
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

## Generate Dataset

Generate 10000 data from a mixture of gamma two gamma distributions.

In [9]:
N = 10000
pi_true = [0.5, 0.5]
a_true  = [20,  80 ]
B_true  = [20,  40 ]

mix_gamma = tfp.distributions.MixtureSameFamily(
    mixture_distribution=tfp.distributions.Categorical(probs=pi_true),
    components_distribution=tfp.distributions.Gamma(concentration=a_true, rate=B_true))

x = mix_gamma.sample(N)

## Variational Inference Under the Shape-Mean Parameterisation

The defualt parameterisation for the function `mix_gamma_vi` is the mean-shape parameterisation under which the variational approximations to the posterior are

$$ q^*(\mathbf{\pi}) = \mathrm{Dirichlet} \left( \zeta_1, ..., \zeta_K \right) ,   $$
$$ q^*(\alpha_k) = \mathcal{N}(\hat{\alpha}_k, \sigma_j^2) .  $$
$$ q^* (\mu_k) =  \operatorname{Inv-Gamma} \left( \gamma_k, \lambda_k \right) ,   $$
The product approximates the joint posterior
$$ p(\mathbf{\pi}, \mathbf{\alpha}, \mathbf{\mu} \mid \mathbf{x}) \approx q^*(\mathbf{\pi}) \prod_{k=1}^K q^*(\alpha_k) q^*(\mu_k). $$

In [10]:
# Fit a model
fit = mix_gamma_vi(x, 2)

# Get the fitted distribution
distribution = fit.distribution()

# Get the means of the parameters under the fitted posterior
distribution.mean()

{'pi': <tf.Tensor: id=2672, shape=(1, 2), dtype=float32, numpy=array([[0.49894944, 0.5010506 ]], dtype=float32)>,
 'mu': <tf.Tensor: id=2679, shape=(1, 2), dtype=float32, numpy=array([[0.9940482, 1.9919776]], dtype=float32)>,
 'alpha': <tf.Tensor: id=2683, shape=(1, 2), dtype=float32, numpy=array([[20.416248, 78.11654 ]], dtype=float32)>}

In [11]:
distribution.stddev()

{'pi': <tf.Tensor: id=2693, shape=(1, 2), dtype=float32, numpy=array([[0.00499924, 0.00499924]], dtype=float32)>,
 'mu': <tf.Tensor: id=2706, shape=(1, 2), dtype=float32, numpy=array([[0.00311455, 0.003184  ]], dtype=float32)>,
 'alpha': <tf.Tensor: id=2710, shape=(1, 2), dtype=float32, numpy=array([[0.4087547, 1.5606917]], dtype=float32)>}

## Variational Inference Under the Shape-Rate Parameterisation

The traditional parameterisation for gamma distribution is the shape-rate parameterisation which this package also supports (although it is not recommended). In this case, the variational approximations to the posterior are

$$ q^*(\mathbf{\pi}) = \mathrm{Dirichlet} \left( \zeta_1, ..., \zeta_K \right) ,   $$
$$ q^*(\alpha_k) = \mathcal{N}(\hat{\alpha}_k, \sigma_k^2) .  $$
$$ q^* (\beta_k) =  \operatorname{Gamma} \left( \gamma_j, \lambda_j \right) ,   $$
The product approximates the joint posterior
$$ p(\mathbf{\pi}, \mathbf{\alpha}, \mathbf{\beta} \mid \mathbf{x}) \approx q^*(\mathbf{\pi}) \prod_{k=1}^K q^*(\alpha_k) q^*(\beta_k) . $$

In [12]:
# Fit a model
fit = mix_gamma_vi(x, 2, parameterisation="shape-rate")

# Get the fitted distribution
distribution = fit.distribution()

# Get the means of the parameters under the fitted posterior
distribution.mean()

{'pi': <tf.Tensor: id=2721, shape=(1, 2), dtype=float64, numpy=array([[0.49918154, 0.50081846]])>,
 'beta': <tf.Tensor: id=2728, shape=(1, 2), dtype=float64, numpy=array([[0.04899512, 0.02543706]])>,
 'alpha': <tf.Tensor: id=2732, shape=(1, 2), dtype=float64, numpy=array([[20.29621189, 78.31683818]])>}

In [13]:
distribution.stddev()

{'pi': <tf.Tensor: id=2742, shape=(1, 2), dtype=float64, numpy=array([[0.00499924, 0.00499924]])>,
 'beta': <tf.Tensor: id=2755, shape=(1, 2), dtype=float64, numpy=array([[1.53936081e-04, 4.06161258e-05]])>,
 'alpha': <tf.Tensor: id=2759, shape=(1, 2), dtype=float64, numpy=array([[0.06292623, 0.12475457]])>}

So, the standard deviation of $\mathbf{\alpha}$ under the shape-rate parameterisation is much lower than it is under the shape-mean parameterisation.